In [13]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 numbers=6,
                 extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.numbers = numbers
        self.extra = extra

    @staticmethod
    def pallottoliere(max_nums):
        while True:
            yield rnd.randint(1, max_nums)

    @staticmethod
    def choice(nums, max_nums):
        def extraction():
            rnd.shuffle(whole_numbers := list(range(1, max_nums+1)))
            single = rnd.choice(whole_numbers)
            whole_numbers.remove(single)

            return single

        return [extraction() for _ in range(nums)]

    @staticmethod
    def sample(nums, max_nums):
        whole_numbers = list(range(1, max_nums+1))

        return rnd.sample(whole_numbers, k=nums)

    def int(self, nums, max_nums):
        combo = set()
        while len(combo) < nums:
            combo.add(next(self.pallottoliere(max_nums)))

        return combo

    @staticmethod
    def many_(backend, nums, max_nums):
        samples = [backend(nums, max_nums) for _ in range(max_nums)]
        rnd.shuffle(samples)

        return rnd.choice(samples)

    @property
    def backend(self):
        return self._backend

    @property
    def many(self):
        return self._many

    def __call__(self, backend='sample', many=True):
        self._backend = backend
        self._many = many

        backend = eval(backend,
                       {'__builtins__': {}},
                       {'choice': self.choice, 'int': self.int, 'sample': self.sample})

        if self._many:
            combo = self.many_(backend,
                               self.numbers,
                               self.max_numbers)

            if self.extra:
                extra = self.many_(backend,
                                   self.extra,
                                   self.max_extra)

        else:
            combo = backend(self.numbers, self.max_numbers)

            if self.extra:
                extra = backend(self.extra, self.max_extra)

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nNumeri Estratti:', *sorted(combo))
        if self.extra:
            print('Superstar:', *sorted(extra))  # type: ignore

In [14]:
eurojack = Lotteria(max_numbers=50, max_extra=12, numbers=5, extra=2)

In [15]:
for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=False)
    print('backend:', eurojack.backend)

print('\n################# many ####################\n')

for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=True)
    print('backend:', eurojack.backend)

######
Estrazione del: 04/06/2023 19:18 
Numeri Estratti: 3 11 23 38 46
Superstar: 4 9
backend: choice
######
Estrazione del: 04/06/2023 19:18 
Numeri Estratti: 9 11 14 17 32
Superstar: 10 11
backend: int
######
Estrazione del: 04/06/2023 19:18 
Numeri Estratti: 7 23 26 42 43
Superstar: 1 5
backend: sample

################# many ####################

######
Estrazione del: 04/06/2023 19:18 
Numeri Estratti: 7 10 12 21 22
Superstar: 7 11
backend: choice
######
Estrazione del: 04/06/2023 19:18 
Numeri Estratti: 9 18 19 32 49
Superstar: 6 11
backend: int
######
Estrazione del: 04/06/2023 19:18 
Numeri Estratti: 2 4 18 25 45
Superstar: 3 5
backend: sample


In [16]:
superenalotto = Lotteria(max_numbers=90, max_extra=90, numbers=6, extra=0)

In [17]:
superenalotto()

Estrazione del: 04/06/2023 19:18 
Numeri Estratti: 11 13 32 35 43 58


In [18]:
superenalotto.extra=1
superenalotto(backend='choice', many=False)

Estrazione del: 04/06/2023 19:18 
Numeri Estratti: 5 19 38 42 74 79
Superstar: 80


In [19]:
superenalotto(backend='int')

Estrazione del: 04/06/2023 19:18 
Numeri Estratti: 4 15 16 24 80 86
Superstar: 45


In [20]:
superenalotto.extra

1

In [21]:
superenalotto.many

True

In [22]:
winForLife = Lotteria(20, 20, 10, 1)
winForLife(backend='int', many=False)

Estrazione del: 04/06/2023 19:18 
Numeri Estratti: 1 2 4 6 7 9 12 13 15 20
Superstar: 14


In [23]:
# trying to set backend via attribute: fails as expected
winForLife.backend = 'choice'

AttributeError: property 'backend' of 'Lotteria' object has no setter